In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
spark.read.table('nyc_yellow.bronzee.nyc_yellow').limit(5).display()

In [0]:
bronze_df = spark.read.table('nyc_yellow.bronzee.nyc_yellow')

# Deduplication part
window_df = Window.partitionBy(
    "VendorID",
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
).orderBy(col("ingested_at").desc())


silver_df = bronze_df.withColumn("rn", row_number().over(window_df))\
    .filter(col("rn") == 1)\
        .drop("rn")

In [0]:
silver_df.createOrReplaceTempView("silver_source")

**MERGE INTO does NOT auto-create the target table.**
**The target table must already exist.**

In [0]:
%sql

CREATE TABLE IF NOT EXISTS nyc_yellow.silver.nyc_yellow_silver
USING DELTA
AS
SELECT * FROM nyc_yellow.bronzee.nyc_yellow
WHERE 1 = 0

In [0]:
%sql

MERGE INTO nyc_yellow.silver.nyc_yellow_silver T
USING silver_source S
ON T.VendorID = S.VendorID 
AND T.tpep_pickup_datetime = S.tpep_pickup_datetime 
AND T.tpep_dropoff_datetime = S.tpep_dropoff_datetime 
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
